# <span style="color:#6042f5"><b>Preprocessing</b>
I will 3 kaggle datasets and try to combine them into one, also will add some random user data like country from which he is(for simulation purpose) etc.
1) https://www.kaggle.com/datasets/tamber/steam-video-games (dataset will steam users and their games and hour-play for each game)
2) https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam (dataset with steam games containing nicely formatted platform that games are on, really tidy data, no need of much preprocessing)
3) https://www.kaggle.com/datasets/nikatomashvili/steam-games-dataset (dataset with steam games containing all info about them, not tidy data need a lot of preprocessing)

In [55]:
import pandas as pd
import string

In [56]:
users = pd.read_csv('datasets/raw_data/steam-200k.csv', sep=';', dtype={
    'user-id': int,
    'hours_played': float,
    'name':str
})
games_platform = pd.read_csv('datasets/raw_data/games.csv', dtype={
    'title':str
})
games_general = pd.read_csv('datasets/raw_data/merged_data.csv', dtype={
    'Title':str
}, header=0, names=['title','og_price','dis_price','rel_date','link','desc','recent_review','all_reviews','recent_review_count','all_review_count','developer','publisher','supported_languages','tags','game_features','pc_specs'])

In [57]:
users.head(2)

,user_id,name,state,hours_played
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,53875128,140,purchase,1.0


In [58]:
games_platform.head(2)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True


In [59]:
games_general.head(2)

,title,og_price,dis_price,rel_date,link,desc,recent_review,all_reviews,recent_review_count,all_review_count,developer,publisher,supported_languages,tags,game_features,pc_specs
0,Baldur's Gate 3,$29.99,$29.99,"3 Aug, 2023",https://store.steampowered.com/app/1086940/Bal...,"Baldur’s Gate 3 is a story-rich, party-based R...",Overwhelmingly Positive,Very Positive,"- 96% of the 128,900 user reviews in the last ...","- 94% of the 188,617 user reviews for this gam...",Larian Studios,Larian Studios,"['English', 'French', 'German', 'Spanish - Spa...","['RPG', 'Choices Matter', 'Character Customiza...","['Single-player', 'Online Co-op', 'LAN Co-op',...",Requires a 64-bit processor and operating syst...
1,Counter-Strike: Global Offensive,$14.99,$14.99,"21 Aug, 2012",https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive (CS: GO) expa...,Very Positive,Very Positive,"- 89% of the 75,284 user reviews in the last 3...","- 88% of the 7,428,921 user reviews for this g...","Valve, Hidden Path Entertainment",Valve,"['English', 'Czech', 'Danish', 'Dutch', 'Finni...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['Steam Achievements', 'Full controller suppor...",OS: | Windows® 7/Vista/XP | Processor: | Int...


In [60]:
print(f"Size of users data set:{users.shape[0]}",
      f"Unique users: {len(set(users.user_id))}",
      f"Size of two game dataset: {games_platform.shape[0]} & {games_general.shape[0]}", sep='\n')

Size of users data set:200000
Unique users: 12393
Size of two game dataset: 50872 & 71700


In [61]:
import unicodedata
allowed_symbols = ":()[]{}-_+=,.!?;"
clean_utf8 = lambda text: ''.join( char for char in str(text) if char.isalnum() or char.isspace() or char in allowed_symbols)

users['name'] = users['name'].apply(clean_utf8)
games_platform['title'] = games_platform['title'].apply(clean_utf8)
games_general['title'] = games_general['title'].apply(clean_utf8)

In [62]:
games_merged = pd.merge(left=games_platform, right=games_general, how='outer', on='title')
games_merged.drop(['og_price','dis_price','rel_date','link','recent_review','all_reviews','recent_review_count','all_review_count','publisher','pc_specs','app_id','date_release','rating','positive_ratio','user_reviews','price_final','price_original','discount'],inplace=True,axis=1)
games_merged.head(5)
# print("\n=======After=======")
# print(f"Size of users data set: {users.shape[0]}",
#       f"Unique users: {len(list(set(users.user_id.values)))}",
#       f"Unique games: {len(set(users.name.values))}",
#       f"Size of merged game dataset: {games_merged.shape[0]}", sep='\n')

,title,win,mac,linux,steam_deck,desc,developer,supported_languages,tags,game_features
0,,NaN,NaN,NaN,NaN,"*** is a small game. It is easy to play,hope y...",Kenshin Game Studio,"['English', 'French', 'Italian', 'German', 'Sp...","['Indie', 'Casual', 'Puzzle']",['Single-player']
1,-circle triangle square-,NaN,NaN,NaN,NaN,"Puzzle game using three types of objects (○, △...","Plucre Co., Ltd.","['English', 'Japanese']","['Casual', 'Puzzle', 'Physics', 'Relaxing', '2...",['Single-player']
2,Circles,True,True,False,True,NaN,NaN,NaN,NaN,NaN
3,Fallalypse,True,True,True,True,A group of terrorists has arranged a nuclear h...,"Hede, Fallalypse studio","['English', 'Japanese', 'Russian', 'Traditiona...","['Early Access', 'Action', 'Adventure', 'Indie...","['Single-player', 'Online PvP', 'Steam Achieve..."
4,Fallalypse Death or Cress,True,True,True,True,The communication with SH42 looking for new pl...,"Hede, Fallalypse studio","['English', 'French', 'German', 'Spanish - Spa...","['Action', 'Adventure', 'Indie', 'RPG', 'Early...","['Single-player', 'Steam Achievements', 'Full ..."


In [63]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

games_merged['desc'] = games_merged['desc'].fillna('')
games_merged['supported_languages'] = games_merged['supported_languages'].fillna('[]')
games_merged['developer'] = games_merged['developer'].fillna('')
games_merged['tags'] = games_merged['tags'].fillna('[]')
games_merged['game_features'] = games_merged['game_features'].fillna('[]')

games_merged['desc'] = games_merged['desc'].apply(remove_punctuation)
games_merged['developer'] = games_merged['developer'].apply(remove_punctuation)

games_merged['game_features'] = games_merged['game_features'].astype('object')
games_merged['tags'] = games_merged['tags'].astype('object')
games_merged['supported_languages'] = games_merged['supported_languages'].astype('object')

games_merged

,title,win,mac,linux,steam_deck,desc,developer,supported_languages,tags,game_features
0,,NaN,NaN,NaN,NaN,is a small game It is easy to playhope you fi...,Kenshin Game Studio,"['English', 'French', 'Italian', 'German', 'Sp...","['Indie', 'Casual', 'Puzzle']",['Single-player']
1,-circle triangle square-,NaN,NaN,NaN,NaN,Puzzle game using three types of objects ○ △ a...,Plucre Co Ltd,"['English', 'Japanese']","['Casual', 'Puzzle', 'Physics', 'Relaxing', '2...",['Single-player']
2,Circles,True,True,False,True,,,[],[],[]
3,Fallalypse,True,True,True,True,A group of terrorists has arranged a nuclear h...,Hede Fallalypse studio,"['English', 'Japanese', 'Russian', 'Traditiona...","['Early Access', 'Action', 'Adventure', 'Indie...","['Single-player', 'Online PvP', 'Steam Achieve..."
4,Fallalypse Death or Cress,True,True,True,True,The communication with SH42 looking for new pl...,Hede Fallalypse studio,"['English', 'French', 'German', 'Spanish - Spa...","['Action', 'Adventure', 'Indie', 'RPG', 'Early...","['Single-player', 'Steam Achievements', 'Full ..."
...,...,...,...,...,...,...,...,...,...,...
86340,잭과 콩나무: 숨겨진 이야기,True,False,False,True,,,[],[],[]
86341,집착의 망자 - 집으로부터의 탈출,True,False,False,True,,,[],[],[]
86342,청춘향전 Cheongchunhyang Jeon,True,False,False,True,This is a drama with multiple endings where Ch...,J Wonder CoLtd,"['English', 'Korean', 'Spanish - Latin America']","['Simulation', 'Cinematic', 'Choices Matter', ...","['Single-player', 'Steam Achievements']"
86343,학생들의 공포괴담 終 (Students horrible stories FIN),True,False,False,True,Students horrible stories FIN is a horror game...,Alman,"['English', 'Korean']","['Indie', 'Visual Novel', 'Horror', 'Simulatio...","['Single-player', 'Steam Achievements']"


In [64]:
users.loc[users['state'] == 'purchase', 'hours_played'] = 0
users.sort_values(by=['user_id','name'])

,user_id,name,state,hours_played
4274,5250,Alien Swarm,purchase,0.0
4275,5250,Alien Swarm,play,4.9
29435,5250,Cities Skylines,purchase,0.0
29436,5250,Cities Skylines,play,144.0
33231,5250,Counter-Strike,purchase,0.0
...,...,...,...,...
139960,309812026,Robocraft,purchase,0.0
55396,309824202,Dota 2,purchase,0.0
55397,309824202,Dota 2,play,0.7
61128,309903146,Dota 2,purchase,0.0


In [65]:
users = users.groupby(by=['user_id', 'name'], as_index=False).agg({
    'hours_played': 'sum',
})
users = users.groupby('user_id',as_index=False).agg(list)
users

,user_id,name,hours_played
0,5250,"[Alien Swarm, Cities Skylines, Counter-Strike,...","[4.9, 144.0, 0.0, 0.0, 0.0, 0.0, 62.0, 0.2, 0...."
1,76767,"[Age of Empires II HD Edition, Alien Swarm, Ar...","[13.1, 0.8, 0.0, 0.0, 0.0, 24.0, 22.0, 12.5, 6..."
2,86540,"[Age of Empires II HD Edition, Age of Empires ...","[0.7, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 57.0, 0.0,..."
3,103360,"[Counter-Strike, Counter-Strike Condition Zero...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,144736,"[Counter-Strike, Day of Defeat, Deathmatch Cla...","[0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...
12388,309554670,[Mitos.is The Game],[5.9]
12389,309626088,[Age of Empires II HD Edition],[6.7]
12390,309812026,"[Counter-Strike Nexon Zombies, Robocraft]","[0.0, 0.0]"
12391,309824202,[Dota 2],[0.7]


In [66]:
users.to_csv('datasets/processed_data/users.csv', index=False, columns=users.columns)
games_merged.to_csv('datasets/processed_data/games.csv', index=False, columns=games_merged.columns)